# Project Title
Sentiment analysis on twitter with nltk. 

## Members

Betül Durkaya, durkaya@itu.edu.tr

## Description of the project
Sentiment analysis the measurement of positivity and negativity of opinions about a topic. Also, NLTK(natural language toolkit) is a tool including libraries and programs for natural language processing in python.

### The methods to be used

Python is going to be used for the project. For access to twitter, we must obtain some keys by help twitter API. We are going to request with codes to connect twitter; furthermore, the connection is going to be actualized owing to tweepy library and aceess keys. In addition, we are going to used json library; accordingly, the response from twitter is in json format. In the beginning, the json data is going to be collected in a txt file. Then, we are going to save the data in a database(MongoDB). After that, we are going to analyzed the data by using a python library that is called NLTK.


### The data

The data is tweets from twitter. Tweets are going to gather in json format.

## Code


In [1]:
import tweepy
from tweepy import OAuthHandler
import listening

consumer_key = "dtKTW35ISxLtPDCszilhNk6R2"
consumer_secret = "pDoisuGvzTGYte5rNEtjguiixVyys0IrL98xu5tzxE7aG8KZCJ"

access_token = "773843946-LBFza4JmUOrzrUOkahDdtHXi6AHwFQyjwH6j7Ibg"
access_token_secret = "JU0ExxNVFkQbBzipW0EOL9Lkb6dm3RadGdfoJeaLdOUgm"

auth = OAuthHandler(consumer_key, consumer_secret)
print("First auth done")
auth.set_access_token(access_token, access_token_secret)
print("2nd auth done")

WORDS = ['iphone']

listener = listening.StreamListener(api=tweepy.API(wait_on_rate_limit=True))
streamer = tweepy.Stream(auth=auth, listener=listener)

print("Tracking: " + str(WORDS))
streamer.filter(track=WORDS, languages=["en"])

First auth done
2nd auth done
Tracking: ['iphone']
Tweet streaming begin.
{'created_at': 'Sun Apr 02 17:02:41 +0000 2017', 'id': 848581461854867457, 'id_str': '848581461854867457', 'text': "RT @errr_ash: I'm not using my phone properly https://t.co/RlFf8Sw3r9", 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 244059161, 'id_str': '244059161', 'name': 'Young Sinatra', 'screen_name': 'Mali_Duma', 'location': 'Durban, South Africa', 'url': None, 'description': 'SC & IG: mali_duma || A Creative Soul', 'protected': False, 'verified': False, 'followers_count': 1108, 'friends_count': 619, 'listed_count': 38, 'favourites_count': 643, 'statuses_count': 39477, 'created_at': 'Fri Jan 28 12:23:36 +0000 2011', 'utc_offset': -7200, 'time_zone': 'Greenland', 'ge

KeyboardInterrupt: 

In [ ]:
from pymongo import MongoClient
import tweepy
import json

connection = MongoClient('localhost', 27017)
db = connection.StreamData
db.tweets.ensure_index("id", unique=True, dropDups=True)
collection = db.tweets


class StreamListener(tweepy.StreamListener):

    def on_connect(self):
        print("Tweet streaming begin.")

    def on_error(self, status):
        print('Error Type: ' + status)
        return False

    def on_data(self, raw):
        data = json.loads(raw)
        tweet_id = data['id_str']  # The ID of tweet from Twitter in string format
        username = data['user']['screen_name']  # The Tweet author's username
        text = data['text']  # The entire body of the Tweet

        try:
            print(data, '\n', (data["user"]["screen_name"], data["text"]), '\n\n\n\n')
            if data["text"].find('RT @') is -1:
                tweet = {'id': tweet_id, 'username': username, 'text': text}
                # Insert Tweet data to MongoDB
                collection.save(tweet)
        except Exception as e:
                    print(e)
        return True